# Configure Code to run on GPU
* Do _not_ use GPU 0; other experiements may accidentally take over

In [1]:
import tensorflow as tf
import os
tf.debugging.set_log_device_placement(False) # displays which devices operations are assigned to

In [2]:
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available: 8


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [3]:
from functions.tf_config import config_gpu
gpu_index = 5               # change this value if desired gpu is occupied
config_gpu(gpu_index, True) # sets desired GPU as the only visible GPU for operations

8 Physical GPUs, 1 Logical GPU
Configured to run on GPU 5


In [4]:
try:
    # specify which device to use
    with tf.device(f"GPU:{gpu_index}"):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)

except RuntimeError as e:
    print(e)
    
print(c)
print("\nPID: ", os.getpid())
# for some reason, log shows GPU 0 but nvidia-smi shows the correct GPU
# suspect that log is displaying a virtual/ logical GPU running _on_ correct GPU

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)

PID:  36678


# Run MNIST Experiment on GPU

In [5]:
from tensorflow import keras
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, xtest = x_train / 255.0, x_test / 255.0

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
])

In [7]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.03739968,  0.6315719 ,  0.40611506,  0.29852355,  0.21687645,
        -0.16408855,  0.04849717,  0.10730891,  0.09384069, -0.00872445]],
      dtype=float32)

In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.08571582, 0.15527685, 0.12393445, 0.11129244, 0.10256679,
        0.07007381, 0.08667235, 0.09192257, 0.09069284, 0.08185205]],
      dtype=float32)

In [9]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

2.658206

In [10]:
model.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2919 - accuracy: 0.9135
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1425 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1069 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0897 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0748 - accuracy: 0.9765


In [12]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 11.0385 - accuracy: 0.9793


[11.038515090942383, 0.9793000221252441]

In [13]:
probability_model = keras.Sequential([
    model,
    keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>